In [4]:
import pandas as pd
import numpy as np

In [5]:
def check(df):
  num_company = df['isin'].nunique()
  num_rows = df.shape[0]
  print(f'Number of companies: {num_company}')
  print(f'Number of rows: {num_rows}')

In [6]:
def get_num_finance(df):
  return df[['isin', 'is_finance']].drop_duplicates()['is_finance'].sum()

# Check the WRDS balance sheet data

In [7]:
balance_sheet = pd.read_csv('2.16 balance sheet.csv')

In [8]:
balance_sheet.head()

,gvkey,fqtr,fyearq,datadate,actq,ancq,atq,lltq,ltq,isin,conm,gind,loc
0,18075,4,2010,2010-12-31,197.081,1026.874,1223.955,707.319,1044.091,BRAZULACNPR4,AZUL SA,203020.0,BRA
1,18075,4,2011,2011-12-31,495.416,1468.587,1964.003,1322.385,1905.621,BRAZULACNPR4,AZUL SA,203020.0,BRA
2,18075,1,2012,2012-03-31,NaN,NaN,NaN,NaN,NaN,BRAZULACNPR4,AZUL SA,203020.0,BRA
3,18075,4,2012,2012-12-31,1011.983,3739.802,4751.785,2431.790,4400.754,BRAZULACNPR4,AZUL SA,203020.0,BRA
4,18075,1,2013,2013-03-31,926.338,3880.680,4807.018,2514.487,4423.179,BRAZULACNPR4,AZUL SA,203020.0,BRA


In [9]:
check(balance_sheet)

Number of companies: 735
Number of rows: 40393


In [10]:
np.sort(balance_sheet['fyearq'].unique())

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], dtype=int64)

In [11]:
# remove the 2005 and 2024 data
balance_sheet_2006 = balance_sheet[~balance_sheet['fyearq'].isin([2005, 2024])]

In [12]:
np.sort(balance_sheet_2006['fyearq'].unique())

array([2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2020, 2021, 2022, 2023], dtype=int64)

In [13]:
check(balance_sheet_2006)

Number of companies: 735
Number of rows: 40356


# Deal with missing values in `balance_sheet_2006`

In [14]:
np.sum(balance_sheet_2006.isnull(), axis = 0)

gvkey           0
fqtr            0
fyearq          0
datadate        0
actq        11747
ancq        11932
atq          1374
lltq        12776
ltq          1386
isin            0
conm            0
gind          980
loc             0
dtype: int64

In [15]:
# remove actq, ancq, lltq
balance_sheet_2006.drop(columns = ['actq', 'ancq', 'lltq'], inplace = True)

C:\Users\thinkpad\AppData\Local\Temp\ipykernel_24868\1928271263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_2006.drop(columns = ['actq', 'ancq', 'lltq'], inplace = True)


In [16]:
balance_sheet_2006.columns

Index(['gvkey', 'fqtr', 'fyearq', 'datadate', 'atq', 'ltq', 'isin', 'conm',
       'gind', 'loc'],
      dtype='object')

In [17]:
balance_sheet_2006.columns = ['key', 'fiscal_quarter', 'fiscal_year', 'datadate', 'total_assets', 'total_liabilities', 'isin', 'company_name', 'gic_industries', 'iso_country_code']

In [18]:
balance_sheet_2006.head(1)

,key,fiscal_quarter,fiscal_year,datadate,total_assets,total_liabilities,isin,company_name,gic_industries,iso_country_code
0,18075,4,2010,2010-12-31,1223.955,1044.091,BRAZULACNPR4,AZUL SA,203020.0,BRA


In [19]:
balance_sheet_2006['gic_industries'] = balance_sheet_2006['gic_industries'].astype(str)

C:\Users\thinkpad\AppData\Local\Temp\ipykernel_24868\2738383306.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_2006['gic_industries'] = balance_sheet_2006['gic_industries'].astype(str)


# Change industry name

In [21]:
gic_codes_table = pd.read_excel('gic industry codes.xlsx')

In [22]:
gic_codes_table.head()

,Sector,Sector2,Industry Group,Industry Group2,Industry,Industry2,Sub-Industry,Sub-Industry2
0,10,Energy,1010,Energy,101010.0,Energy Equipment & Services,10101010.0,Oil & Gas Drilling
1,10,Energy,1010,Energy,101010.0,Energy Equipment & Services,10101020.0,Oil & Gas Equipment & Services
2,10,Energy,1010,Energy,101020.0,"Oil, Gas & Consumable Fuels",10102010.0,Integrated Oil & Gas
3,10,Energy,1010,Energy,101020.0,"Oil, Gas & Consumable Fuels",10102020.0,Oil & Gas Exploration & Production
4,10,Energy,1010,Energy,101020.0,"Oil, Gas & Consumable Fuels",10102030.0,Oil & Gas Refining & Marketing


In [23]:
gic_codes_table = gic_codes_table[['Industry', 'Industry2']]
gic_codes_table.columns = ['industry_code', 'industry_name']
gic_codes_table.drop_duplicates(inplace = True)
gic_codes_table['industry_code'] = gic_codes_table['industry_code'].astype(str)
gic_codes_table.set_index('industry_code', inplace = True)
gic_codes_table.head()

,industry_name
industry_code,
101010.0,Energy Equipment & Services
101020.0,"Oil, Gas & Consumable Fuels"
151010.0,Chemicals
151020.0,Construction Materials
151030.0,Containers & Packaging


In [24]:
def get_industry_name(df):
  code = df['gic_industries']
  series = gic_codes_table['industry_name']
  return series[code]

In [25]:
balance_sheet_2006['industry_name'] = balance_sheet_2006.apply(get_industry_name, axis = 1)

C:\Users\thinkpad\AppData\Local\Temp\ipykernel_24868\181643997.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_2006['industry_name'] = balance_sheet_2006.apply(get_industry_name, axis = 1)


In [26]:
balance_sheet_2006.head(1)

,key,fiscal_quarter,fiscal_year,datadate,total_assets,total_liabilities,isin,company_name,gic_industries,iso_country_code,industry_name
0,18075,4,2010,2010-12-31,1223.955,1044.091,BRAZULACNPR4,AZUL SA,203020.0,BRA,Passenger Airlines


In [27]:
balance_sheet_2006['is_finance'] = balance_sheet_2006['gic_industries'].str.startswith('40')
get_num_finance(balance_sheet_2006)

C:\Users\thinkpad\AppData\Local\Temp\ipykernel_24868\1789324094.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_2006['is_finance'] = balance_sheet_2006['gic_industries'].str.startswith('40')


109

In [28]:
balance_sheet_2006[balance_sheet_2006['is_finance'] == True]['industry_name'].unique()

array(['Financial Services', 'Banks', 'Capital Markets', 'Insurance',
       'Consumer Finance',
       'Mortgage Real Estate Investment Trusts (REITs)'], dtype=object)

# Check the industries

In [29]:
balance_sheet_2006[['isin', 'industry_name']].drop_duplicates()['industry_name'].value_counts()

Food Products          48
Banks                  38
Household Durables     38
Capital Markets        37
Metals & Mining        36
                       ..
Aerospace & Defense     1
Retail REITs            1
Office REITs            1
Distributors            1
Specialized REITs       1
Name: industry_name, Length: 67, dtype: int64

In [30]:
np.sum(balance_sheet_2006.isnull(), axis = 0)

key                     0
fiscal_quarter          0
fiscal_year             0
datadate                0
total_assets         1374
total_liabilities    1386
isin                    0
company_name            0
gic_industries          0
iso_country_code        0
industry_name         980
is_finance              0
dtype: int64

# Remove the companies with no industry data

In [31]:
balance_sheet_2006 = balance_sheet_2006[balance_sheet_2006['industry_name'].notnull()]
check(balance_sheet_2006)

Number of companies: 697
Number of rows: 39376


In [32]:
np.sum(balance_sheet_2006.isnull(), axis = 0)

key                     0
fiscal_quarter          0
fiscal_year             0
datadate                0
total_assets         1315
total_liabilities    1326
isin                    0
company_name            0
gic_industries          0
iso_country_code        0
industry_name           0
is_finance              0
dtype: int64

In [33]:
get_num_finance(balance_sheet_2006)

109

## Check the starting date of each company

In [34]:
balance_sheet_2006['datadate'] = pd.to_datetime(balance_sheet_2006['datadate'])
balance_sheet_2006['MM-DD'] = balance_sheet_2006['datadate'].dt.strftime('%m-%d')

In [35]:
balance_sheet_2006['MM-DD'].value_counts()

12-31    9797
03-31    9585
06-30    9555
09-30    9547
08-31     143
05-31     127
11-30     124
02-28     103
07-31      94
10-31      94
04-30      90
01-31      89
02-29      28
Name: MM-DD, dtype: int64

In [36]:
balance_sheet_2006['MM-DD'].value_counts().index

Index(['12-31', '03-31', '06-30', '09-30', '08-31', '05-31', '11-30', '02-28',
       '07-31', '10-31', '04-30', '01-31', '02-29'],
      dtype='object')

### Decision:
Drop unsuitable rows.

In [37]:
balance_sheet_2006 = balance_sheet_2006[~(balance_sheet_2006['MM-DD'].isin(['08-31', '05-31', '11-30', '10-31', '02-28', '07-31', '01-31', '04-30', '02-29']))]

In [38]:
check(balance_sheet_2006)

Number of companies: 680
Number of rows: 38484


In [39]:
balance_sheet_2006['MM-DD'].value_counts()

12-31    9797
03-31    9585
06-30    9555
09-30    9547
Name: MM-DD, dtype: int64

# Change date columns + interpolation

In [40]:
quarterly_dates = pd.date_range(start='2006-01-01', end='2023-12-31', freq='Q').to_list()
correct_datetime = pd.DataFrame({'datadate': quarterly_dates})
correct_datetime.head()

,datadate
0,2006-03-31
1,2006-06-30
2,2006-09-30
3,2006-12-31
4,2007-03-31


In [41]:
# find the expected length of the dataframe
expected_length = len(quarterly_dates*(balance_sheet_2006['isin'].nunique()))
expected_length

48960

In [42]:
correct_datetime.set_index('datadate', inplace = True)

In [43]:
company_list = list(balance_sheet_2006['isin'].unique())

In [44]:
check(balance_sheet_2006)

Number of companies: 680
Number of rows: 38484


In [45]:
np.sum(balance_sheet_2006.isnull(), axis = 0)

key                     0
fiscal_quarter          0
fiscal_year             0
datadate                0
total_assets         1248
total_liabilities    1260
isin                    0
company_name            0
gic_industries          0
iso_country_code        0
industry_name           0
is_finance              0
MM-DD                   0
dtype: int64

In [46]:
first_company_info = balance_sheet_2006[balance_sheet_2006['isin'] == company_list[0]].set_index('datadate')
balance_sheet_adjusted = first_company_info.join(correct_datetime, how = 'outer').reset_index()

for column in ['key', 'isin', 'company_name', 'gic_industries', 'iso_country_code']:
    balance_sheet_adjusted[column] = balance_sheet_adjusted[balance_sheet_adjusted[column].notnull()][column].unique().item()

balance_sheet_adjusted['fiscal_quarter']=balance_sheet_adjusted['datadate'].dt.quarter
balance_sheet_adjusted['fiscal_year']=balance_sheet_adjusted['datadate'].dt.year

# interpolation:
balance_sheet_adjusted.set_index('datadate', inplace = True)
balance_sheet_adjusted['total_assets'] = balance_sheet_adjusted['total_assets'].interpolate(method='time', limit_direction='both')
balance_sheet_adjusted['total_liabilities'] = balance_sheet_adjusted['total_liabilities'].interpolate(method='time', limit_direction='both')
balance_sheet_adjusted.shape
balance_sheet_adjusted.reset_index(drop = False, inplace = True)

In [47]:
questionable_isin_dict = {}

for isin in company_list[1:]:
    temp = balance_sheet_2006[balance_sheet_2006['isin'] == isin].set_index('datadate')
    temp = temp.join(correct_datetime, how = 'outer').reset_index()
    if temp.shape[0] != 72:
      print(isin, list(temp[temp['datadate'].duplicated()]['datadate']))
      questionable_isin_dict = {**questionable_isin_dict, **{isin: list(temp[temp['datadate'].duplicated()]['datadate'])}}

    for column in ['key', 'isin', 'company_name', 'gic_industries', 'iso_country_code', 'industry_name', 'is_finance']:
      temp[column] = temp[temp[column].notnull()][column].unique().item()

    temp['fiscal_quarter']=temp['datadate'].dt.quarter
    temp['fiscal_year']=temp['datadate'].dt.year

    # interpolation:
    temp.set_index('datadate', inplace = True)
    temp['total_assets'] = temp['total_assets'].interpolate(method='time', limit_direction='both')
    temp['total_liabilities'] = temp['total_liabilities'].interpolate(method='time', limit_direction='both')
    temp.reset_index(drop = False, inplace = True)

    balance_sheet_adjusted = pd.concat([balance_sheet_adjusted, temp], axis = 0)

print(balance_sheet_adjusted.shape[0])

ARP7905G1652 [Timestamp('2007-03-31 00:00:00'), Timestamp('2008-03-31 00:00:00')]
BRGSHPACNOR7 [Timestamp('2007-03-31 00:00:00'), Timestamp('2007-06-30 00:00:00'), Timestamp('2007-09-30 00:00:00')]
ARINAG010041 [Timestamp('2022-03-31 00:00:00'), Timestamp('2022-06-30 00:00:00'), Timestamp('2022-09-30 00:00:00')]
JMP004681012 [Timestamp('2015-06-30 00:00:00')]
48969


In [48]:
balance_sheet_adjusted['index'] = balance_sheet_adjusted.reset_index().index
for isin, time_list in questionable_isin_dict.items():
  temp = balance_sheet_adjusted[balance_sheet_adjusted['isin'] == isin]
  temp.head()
  print(temp[temp['datadate'].isin(time_list)].to_string())
  print('--------------')

     datadate       key  fiscal_quarter  fiscal_year  total_assets  total_liabilities          isin company_name gic_industries iso_country_code               industry_name is_finance  MM-DD  index
4  2007-03-31  208536.0               1         2007      260.8090            27.7072  ARP7905G1652   POLLEDO SA       201030.0              ARG  Construction & Engineering      False  03-31   9220
5  2007-03-31  208536.0               1         2007      260.8090            27.7072  ARP7905G1652   POLLEDO SA       201030.0              ARG  Construction & Engineering      False  03-31   9221
9  2008-03-31  208536.0               1         2008      244.0232            33.2560  ARP7905G1652   POLLEDO SA       201030.0              ARG  Construction & Engineering      False  03-31   9225
10 2008-03-31  208536.0               1         2008      244.0232            33.2560  ARP7905G1652   POLLEDO SA       201030.0              ARG  Construction & Engineering      False  03-31   9226
----------

In [49]:
check(balance_sheet_adjusted)

Number of companies: 680
Number of rows: 48969


In [50]:
expected_length

48960

In [51]:
balance_sheet_adjusted.drop(columns = ['MM-DD'], inplace = True)

In [52]:
balance_sheet_adjusted.drop_duplicates(subset = ['isin', 'datadate'], keep = 'last', inplace = True)
check(balance_sheet_adjusted)

Number of companies: 680
Number of rows: 48960


In [53]:
balance_sheet_adjusted.drop(columns = ['index'], inplace = True)

In [54]:
get_num_finance(balance_sheet_adjusted)

108

# Check the rest of null values

In [55]:
check(balance_sheet_adjusted)

Number of companies: 680
Number of rows: 48960


In [56]:
np.sum(balance_sheet_adjusted.isnull(), axis = 0)

datadate              0
key                   0
fiscal_quarter        0
fiscal_year           0
total_assets          0
total_liabilities     0
isin                  0
company_name          0
gic_industries        0
iso_country_code      0
industry_name        28
is_finance           28
dtype: int64

In [57]:
balance_sheet_adjusted['industry_name'] = balance_sheet_adjusted.apply(get_industry_name, axis = 1)
balance_sheet_adjusted['is_finance'] = balance_sheet_adjusted['gic_industries'].str.startswith('40')

In [58]:
np.sum(balance_sheet_adjusted.isnull(), axis = 0)

datadate             0
key                  0
fiscal_quarter       0
fiscal_year          0
total_assets         0
total_liabilities    0
isin                 0
company_name         0
gic_industries       0
iso_country_code     0
industry_name        0
is_finance           0
dtype: int64

In [59]:
check(balance_sheet_adjusted)

Number of companies: 680
Number of rows: 48960


In [60]:
balance_sheet_adjusted.head(1)

,datadate,key,fiscal_quarter,fiscal_year,total_assets,total_liabilities,isin,company_name,gic_industries,iso_country_code,industry_name,is_finance
0,2006-03-31,18075.0,1,2006,1223.955,1044.091,BRAZULACNPR4,AZUL SA,203020.0,BRA,Passenger Airlines,False


In [61]:
get_num_finance(balance_sheet_adjusted)

108

# Export the interpolated balance sheet

In [62]:
#balance_sheet_adjusted.to_csv('2.18 interpolated balance sheet.csv', index=False)

# Export the isin list that will be used for stock data

In [63]:
isin_list = list(balance_sheet_adjusted['isin'].unique())
with open('2.18 isin list.txt', 'w') as f:
    for item in isin_list:
        f.write(f"{item}\n")